# Redlining by County 

The term redlining comes from the practice of the FHA and the Home Owners' Loan Corporation (HOLC)  is color code neighborhood maps in order to identify how safe it was to insure mortgages in certain neighborhoods. Neighborhoods were graded in four categories: A- “Best” (Green), B- “Still Desirable” (Blue), C- “Defiantly Declining” (Yellow), D- “Hazardous” (Red); this classifications were primarily racially motivated, placing neighborhoods where minorities live in C or D classifications.  
In 1968,  the Fair Housing Act was passed, which makes it unlawful to discriminate in terms or conditions in the basis of race or national origin.
And in 1974, The Equal Credit Opportunity Act (ECOA) enacted unlawful for any creditor to discriminate against any applicant, with respect to any aspect of a credit transaction, based on race, color, religion, national origin, sex, marital status, or age
Using HOLC historical redlining information we decided to plot total area (sq ft) redlined against current county lines. 



In [1]:
# Load libraries
import pandas as pd
import numpy as np
import altair as alt
import geopandas as gpd
import pyspark
import censusdata
from pyspark.sql import SparkSession 
from pyspark.sql.functions  import col, when, lit
from pyspark.sql import functions as f

from vega_datasets import data
alt.data_transformers.disable_max_rows()

from io import StringIO
alt.themes.enable("fivethirtyeight") # visualization team 


ThemeRegistry.enable('fivethirtyeight')

### Load dataset HOLC 

Calculate Historical Redlining Score (HRS) by calculating the grade weights. 

undefined. Calculate the percentage of weighted area. Ex. area_A divided by area_rated.

undefined. Multiply by the HOLC grade factor : A= 1, B=2, C=3, D=4

undefined. Final Score. The level of redlining goes from 1-4 with 1 being low redlining and 4 high.

This methodology was obtained from https://ncrc.org/redlining-score/

In [2]:


holc_rated=pd.read_csv('/work/HOLC_2020_census_tracts/HOLC_2020_census_tracts.csv', dtype={'geoid20': str})
#calculate % of rated area
holc_rated['A']= holc_rated['area_A']/holc_rated['area_rated']
holc_rated['B']= holc_rated['area_B']/holc_rated['area_rated']
holc_rated['C']= holc_rated['area_C']/holc_rated['area_rated']
holc_rated['D']= holc_rated['area_D']/holc_rated['area_rated']

#used NCRC methodology to calculate HRS (Historic redlinning score)
holc_rated['a']= holc_rated['A']*1
holc_rated['b']= holc_rated['B']*2
holc_rated['c']= holc_rated['C']*3
holc_rated['d']= holc_rated['D']*4
holc_rated['HRS']= holc_rated[['a', 'b', 'c', 'd']].sum(axis=1)

holc_rated['fips']= holc_rated['geoid20'].str[:5]  #extract county code also known as fips
holc_rated.rename(columns={'geoid20':'GEOID'}, inplace=True)
holc_rated

,GEOID,class1,class1_lbl,class2,class2_lbl,class2_red,class3,class3_lbl,area_total,area_rated,...,A,B,C,D,a,b,c,d,HRS,fips
0,01073000100,D,Mainly D,D-C,"Mainly D, some C","Mainly D, some C",D-C-B,"Mainly D, some C, some B",7549580.5,73.293671,...,0.0,0.058142,0.355992,0.585866,0.0,0.116284,1.067976,2.343463,3.527723,01073
1,01073000300,D,Mainly D,D-C,"Mainly D, some C","Mainly D, some C",D-C,"Mainly D, some C",2078504.4,94.276787,...,0.0,0.000000,0.000430,0.999570,0.0,0.000000,0.001290,3.998280,3.999570,01073
2,01073000400,D,Mainly D,D-C,"Mainly D, some C","Mainly D, some C",D-C,"Mainly D, some C",7998765.0,46.557659,...,0.0,0.000000,0.222244,0.777756,0.0,0.000000,0.666733,3.111022,3.777756,01073
3,01073000500,D,Mainly D,D,Mainly D,Only D,D,Mainly D,4680667.0,64.390160,...,0.0,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,4.000000,4.000000,01073
4,01073000700,D,Mainly D,D,Mainly D,Only D,D,Mainly D,3520562.8,41.288933,...,0.0,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,4.000000,4.000000,01073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15535,55139001100,D,Mainly D,D,Mainly D,Only D,D,Mainly D,2698989.0,29.276098,...,0.0,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,4.000000,4.000000,55139
15536,55139001200,D,Mainly D,D,Mainly D,Only D,D,Mainly D,1514734.6,47.844109,...,0.0,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,4.000000,4.000000,55139
15537,55139001300,D,Mainly D,D-C,"Mainly D, some C","Mainly D, some C",D-C,"Mainly D, some C",3021260.8,27.407440,...,0.0,0.000000,0.428756,0.571244,0.0,0.000000,1.286269,2.284975,3.571244,55139
15538,55139001400,C,Mainly C,C-D,"Mainly C, some D","Mainly C, some D",C-D,"Mainly C, some D",6286801.0,22.926498,...,0.0,0.000000,0.820599,0.179401,0.0,0.000000,2.461798,0.717603,3.179401,55139


### Loading county codes and geolocation.

We would like to explore if redlining was more prevalent in certain areas of the country. For that we will use two more datasets for plotting.

1. https://github.com/btskinner/spatial/blob/master/data/county_centers.csv

undefined. https://github.com/kjhealy/fips-codes/blob/master/state_and_county_fips_master.csv

In [3]:
states=pd.read_csv('/work/state_fips.csv', dtype={'fips': str})
states['fips'] = states['fips'].str.zfill(5)
fips= pd.read_csv('/work/fipsnames-20221011-151647.csv', dtype={'fips': str})
fips= pd.merge(fips[['fips', 'clon00', 'clat00']], states[['fips', 'name']],how='left',on='fips')
fips.head(3)

,fips,clon00,clat00,name
0,01001,-86.577176,32.523283,Autauga County
1,01003,-87.748260,30.592781,Baldwin County
2,01005,-85.331312,31.856515,Barbour County


We will joined the newly created fips file with HOLC to plot counties and percentage of redlining 

In [4]:
holc_fips= holc_rated[['fips', 'A', 'B', 'C', 'D', 'HRS']].groupby('fips').mean()
holc_fips.reset_index(inplace=True)
holc_fips["id"] = holc_fips["fips"].astype(int)
holc_fips= pd.merge(holc_fips, fips,how='left',on='fips')

#Long form for plotting
holc_fipsL= pd.melt(holc_fips, id_vars=['fips', 'id', 'name', 'clon00', 'clat00', 'HRS' ], value_vars=['A', 'B', 'C', 
    'D'], ignore_index=False)

holc_fips.head(3)

,fips,A,B,C,D,HRS,id,clon00,clat00,name
0,01073,0.053820,0.108160,0.221830,0.616191,3.400391,1073,-86.828050,33.527872,Jefferson County
1,01097,0.007314,0.074699,0.353903,0.564083,3.474756,1097,-88.139667,30.722256,Mobile County
2,01101,0.070889,0.113328,0.350049,0.465733,3.210627,1101,-86.260844,32.331872,Montgomery County


We used Altair to plot all counties with redlining, we can see that the majority of redlining counties are in the east coast or near metropolitan areas, all so must of the neighborhoods were classified grade "D'

In [5]:
counties = alt.topo_feature(data.us_10m.url, 'counties')
# US states background
background2 = alt.Chart(counties).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    width=800,
    height=500
).project('albersUsa')

# airport positions on background
range_=['#6d904f', '#30a2da', '#e5ae38', '#fc4f30']
points = alt.Chart(holc_fipsL).mark_circle( opacity=0.8,
       stroke='black',
       strokeWidth=1).encode(
    longitude='clon00:Q',
    latitude='clat00:Q',
    size=alt.Size('value:Q', title='% of Area Rated', scale=alt.Scale(range=[0, 500])  ),
    color= alt.Color('variable',scale=alt.Scale( range=range_,),  title='Grade'),
    tooltip=['name:N','HRS:Q']
).properties(
    title='Historical Redlining by 2021 County Lines'
)

mapCounties=background2 + points
mapCounties.configure(background='#FFFFFF')

/opt/conda/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

### Census Data

Next, we will compare the demographics of ungraded and graded areas. For this we will use census data available to python library Censusdata. Library documentation can be found https://pypi.org/project/CensusData/. 

In [6]:
#dowload census data
county_pop = censusdata.download('acs5', 2015, censusdata.censusgeo([('county', '*')]),
                                ['B02001_001E', 'B02001_002E', 'B25081_001E', 'B25081_008E', 
                                'B25002_001E', 'B25002_002E', 'B25002_003E'])
county_pop.rename(columns={'B02001_001E':'population_total', 'B02001_002E':'white_pop',
  'B25081_001E':'total_houses','B25081_008E':'houses_wo_mortgage','B25002_001E': 'occupancy_total', 
  'B25002_002E': 'occupied', 'B25002_003E': 'Vacant'}, inplace=True)

county_pop.reset_index(inplace=True)
county_pop

,index,population_total,white_pop,total_houses,houses_wo_mortgage,occupancy_total,occupied,Vacant
0,"Childress County, Texas: Summary level: 050, s...",7059,5701,1531,791,3076,2391,685
1,"Comal County, Texas: Summary level: 050, state...",119632,108851,33372,13136,50288,43951,6337
2,"Houston County, Texas: Summary level: 050, sta...",22949,15459,5530,3769,11543,8046,3497
3,"Navarro County, Texas: Summary level: 050, sta...",48118,36943,11624,6191,20431,17477,2954
4,"Scurry County, Texas: Summary level: 050, stat...",17238,14988,4422,2610,7137,5832,1305
...,...,...,...,...,...,...,...,...
3215,"Charlotte County, Virginia: Summary level: 050...",12313,8250,3281,1850,6298,4723,1575
3216,"Burleson County, Texas: Summary level: 050, st...",17293,13613,5135,3285,8875,6401,2474
3217,"Greensville County, Virginia: Summary level: 0...",11760,4646,2495,1270,4122,3486,636
3218,"Thurston County, Washington: Summary level: 05...",262723,216973,66262,19674,110904,102631,8273


Since the census data does not contain the five digit fips code for each county, we will perform some transformation 

In [7]:
#extract state and 3 digit county code. And build fips code. 
county_pop['state']= county_pop['index'].astype(str).str.extract(r'(state:\d{2})')
county_pop['county']= county_pop['index'].astype(str).str.extract(r'(county:\d{3})')
county_pop['county']= county_pop['county'].str.replace("county:", "")
county_pop['state']= county_pop['state'].str.replace("state:", "")
county_pop['fips']= county_pop['state']+ county_pop['county']
county_pop.drop(columns=['state', 'county'], inplace=True)
county_pop

,index,population_total,white_pop,total_houses,houses_wo_mortgage,occupancy_total,occupied,Vacant,fips
0,"Childress County, Texas: Summary level: 050, s...",7059,5701,1531,791,3076,2391,685,48075
1,"Comal County, Texas: Summary level: 050, state...",119632,108851,33372,13136,50288,43951,6337,48091
2,"Houston County, Texas: Summary level: 050, sta...",22949,15459,5530,3769,11543,8046,3497,48225
3,"Navarro County, Texas: Summary level: 050, sta...",48118,36943,11624,6191,20431,17477,2954,48349
4,"Scurry County, Texas: Summary level: 050, stat...",17238,14988,4422,2610,7137,5832,1305,48415
...,...,...,...,...,...,...,...,...,...
3215,"Charlotte County, Virginia: Summary level: 050...",12313,8250,3281,1850,6298,4723,1575,51037
3216,"Burleson County, Texas: Summary level: 050, st...",17293,13613,5135,3285,8875,6401,2474,48051
3217,"Greensville County, Virginia: Summary level: 0...",11760,4646,2495,1270,4122,3486,636,51081
3218,"Thurston County, Washington: Summary level: 05...",262723,216973,66262,19674,110904,102631,8273,53067


We will calculate the vacancy, mortgage and minority percentages

In [8]:
county_pop['vacant_perc']= county_pop['Vacant']/county_pop['total_houses']
county_pop['mortgage_perc']= 1-(county_pop['houses_wo_mortgage']/county_pop['total_houses'])
county_pop['minority_perc']= 1-(county_pop['white_pop']/county_pop['population_total'])
county_pop

,index,population_total,white_pop,total_houses,houses_wo_mortgage,occupancy_total,occupied,Vacant,fips,vacant_perc,mortgage_perc,minority_perc
0,"Childress County, Texas: Summary level: 050, s...",7059,5701,1531,791,3076,2391,685,48075,0.447420,0.483344,0.192379
1,"Comal County, Texas: Summary level: 050, state...",119632,108851,33372,13136,50288,43951,6337,48091,0.189890,0.606377,0.090118
2,"Houston County, Texas: Summary level: 050, sta...",22949,15459,5530,3769,11543,8046,3497,48225,0.632369,0.318445,0.326376
3,"Navarro County, Texas: Summary level: 050, sta...",48118,36943,11624,6191,20431,17477,2954,48349,0.254129,0.467395,0.232242
4,"Scurry County, Texas: Summary level: 050, stat...",17238,14988,4422,2610,7137,5832,1305,48415,0.295115,0.409769,0.130526
...,...,...,...,...,...,...,...,...,...,...,...,...
3215,"Charlotte County, Virginia: Summary level: 050...",12313,8250,3281,1850,6298,4723,1575,51037,0.480037,0.436148,0.329976
3216,"Burleson County, Texas: Summary level: 050, st...",17293,13613,5135,3285,8875,6401,2474,48051,0.481792,0.360273,0.212803
3217,"Greensville County, Virginia: Summary level: 0...",11760,4646,2495,1270,4122,3486,636,51081,0.254910,0.490982,0.604932
3218,"Thurston County, Washington: Summary level: 05...",262723,216973,66262,19674,110904,102631,8273,53067,0.124853,0.703088,0.174138


### Joining redlining and census data 

We will join the HRS with current census data, to see the demographic composition and HRS grading together.  

Minority percentage steadily increases from ungraded to grade D. We can also see that the % of vacant units doubles between graded A areas and Graded D. mortgage percentage is similar between all areas. 

In [9]:
#merge HOLC, and census data
fips_rated= pd.merge(holc_fips, county_pop[['fips', 'population_total', 'total_houses', 'vacant_perc',
    'mortgage_perc', 'minority_perc']],
    how='right',on='fips')
fips_rated['HRS'] = fips_rated['HRS'].fillna(0.1)
fips_rated['grade'] = pd.cut(fips_rated['HRS'], bins=[0,1, 1.75, 2.49, 3.3, 4], labels=['Ungraded', 'A','B', 
    'C', 'D'])

#aggreagate values by HRS grading 
names = {'population_total':'population_total', 'total_houses':'total_houses','mortgage_perc':'mean_%mortage',
     'minority_perc':'mean-%minority', 'vacant_perc':'mean_%vacant', 'HRS':'mean_HRS'}
fips_ratedagg= fips_rated.groupby('grade').agg({'population_total':'sum', 'total_houses':'sum',
    'mortgage_perc':'mean', 'minority_perc':'mean', 'vacant_perc':'mean', 'HRS':'mean' }).rename(columns=names)
fips_ratedagg

,population_total,total_houses,mean_%mortage,mean-%minority,mean_%vacant,mean_HRS
grade,,,,,,
Ungraded,175453047,44401875,0.515968,0.160181,0.372752,0.100000
A,1485506,350708,0.732288,0.251013,0.091065,1.251970
B,12337756,3058909,0.667776,0.223208,0.142555,2.199496
C,120461335,25482747,0.662170,0.269166,0.183875,2.895074
D,10360450,2275562,0.665727,0.314266,0.201965,3.512371


### Loan applications and HRS

Considering that there is significant increase in minority population and vacant units as HRS grade increases. We will explore the loan performance, to see if a relationship exist with HRS. 

In [10]:
#pyspark session to load loan data 


spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName('My First Spark application') \
    .getOrCreate()
sc = spark.sparkContext

In [11]:
df_hm = spark.read.option("header",True) \
     .csv("/work/2021_public_lar.csv")
df_hm.show(2,truncate=False)

+-------------+--------------------+--------------+----------+-----------+------------+---------------------+-------------------------+------------------------------------+----------------------+------------+-----------+------------+--------------+-----------+---------+------------+-----------+----------------+-----------------------+------------------------------+-----------+----------------------------+-------------+-----------+------------+----------------+---------------------+-------------------+---------------+--------------+---------+-----------------------+-----------------+---------------------+---------------------+---------------+----------------------------+--------------+-------------------+--------------+---------------------------------------+----------------------------------------+-----------+----------------------------+------+--------------------+---------------------------+------------------------------+---------------------+---------------------+-------------------

We will focus on single family homes, not used for businesses, we will also exclude any applications that were closed out due to lack of documentation or withdraw by applicant. 

In [12]:
# filter only loans for home purchases and for personal use, etc refer to data cleaning specs. .
df_hm_cleaned = df_hm.select('*')\
    .filter((df_hm.business_or_commercial_purpose == 2) & (df_hm.loan_purpose ==1) &
            (df_hm.occupancy_type ==1)& (df_hm.action_taken !=4) &
            (df_hm.action_taken !=5) & (df_hm.loan_type ==1)&
            (df_hm.derived_dwelling_category == 'Single Family (1-4 Units):Site-Built' )&
            (df_hm.derived_loan_product_type == "Conventional:First Lien") &
            (df_hm.conforming_loan_limit == "C") &
            (df_hm.lien_status == 1) &
            (df_hm.reverse_mortgage == 2) &
            (df_hm.open_end_line_of_credit == 2) &
            (df_hm.negative_amortization == 2 ) &
            (df_hm.total_units == 1)&
            (df_hm.balloon_payment ==2))

Since Hispanic is encoded under 'derived_ethnicity', We will create a variable called race. 

In [13]:
#filter out only by certain races
races=['White', 'Black or African American', 'Asian', 'Hispanic or Latino']
df_hm_cleaned= df_hm_cleaned.withColumn('race', \
    f.when(f.col('derived_ethnicity')=='Hispanic or Latino', "Hispanic or Latino")\
    .otherwise(df_hm_cleaned.derived_race))

df_hm_cleaned =df_hm_cleaned.select('*').filter(df_hm_cleaned.race.isin(races))

We will calculate the total number of applications, approvals and mean interest rate, creating a consolidated pandas data frame. 

In [14]:
count_group=df_hm_cleaned.groupBy('census_tract',"race").count()

approvals=df_hm_cleaned.filter(col('action_taken').isin([1,2,6,8]))\
    .groupBy('census_tract','race').count().withColumnRenamed("count","approved")\
    .withColumnRenamed('race',"race2").withColumnRenamed("census_tract","census")

interest_rate=df_hm_cleaned.groupBy('census_tract',"race").agg(f.mean('interest_rate'))\
    .withColumnRenamed('census_tract','census_tract2').withColumnRenamed("race","race3")

approv_index=count_group.join(approvals,(count_group.race == approvals.race2)\
    & (count_group.census_tract == approvals.census),"left")\
    .join(interest_rate, (count_group.race == interest_rate.race3)\
    & (count_group.census_tract == interest_rate.census_tract2),"left").toPandas()

We will like to add the percentage of approvals to our new data frame 

In [15]:
approv_index.drop(['race2', 'census', 'race3', 'census_tract2'], axis=1, inplace=True)
approv_index['approved'] = approv_index['approved'].fillna(0) #nan in column approved are denials will be transform to 0
approv_index['approval_perc']= approv_index['approved']/approv_index['count']
approv_index

,census_tract,race,count,approved,avg(interest_rate),approval_perc
0,01001020100,Black or African American,1,1.0,2.990000,1.000000
1,01001020100,White,1,1.0,3.500000,1.000000
2,01001020200,White,1,1.0,2.990000,1.000000
3,01001020300,Black or African American,1,1.0,3.250000,1.000000
4,01001020300,White,4,4.0,3.312500,1.000000
...,...,...,...,...,...,...
103872,72153750400,Hispanic or Latino,1,1.0,3.125000,1.000000
103873,NA,Asian,932,598.0,3.056142,0.641631
103874,NA,Black or African American,1981,701.0,3.242254,0.353862
103875,NA,Hispanic or Latino,3849,1834.0,3.377341,0.476487


In [16]:
total_applications= approv_index.groupby('race')['count', ].sum()
total_applications

,count
race,
Asian,40623
Black or African American,25972
Hispanic or Latino,50338
White,337388


Our data frame encodes the variable race in one columns. We will perform some transformation in order to calculate the correlation with HRS. 

In [17]:
ai_long= pd.pivot_table(approv_index, values=['approval_perc', 'avg(interest_rate)'], index=['census_tract'],
                            columns=['race'])
ai_long.columns = ai_long.columns.droplevel()


  
# Set the index
columns_ = ['asian_%approv', 'black_%approv', 'hisp_%approv', 'white_%approv', 'asian_interest', 'black_interest',
     'hisp_interest', 'white_interest']
ai_long.columns = columns_
ai_long.reset_index(inplace=True)
ai_long

,census_tract,asian_%approv,black_%approv,hisp_%approv,white_%approv,asian_interest,black_interest,hisp_interest,white_interest
0,01001020100,NaN,1.000000,NaN,1.000000,NaN,2.990000,NaN,3.500000
1,01001020200,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,2.990000
2,01001020300,NaN,1.000000,NaN,1.000000,NaN,3.250000,NaN,3.312500
3,01001020400,NaN,1.000000,1.000000,1.000000,NaN,3.125000,3.500000,2.708333
4,01001020500,0.500000,1.000000,1.000000,1.000000,2.000000,3.062500,2.990000,3.129750
...,...,...,...,...,...,...,...,...,...
59873,72145560402,NaN,NaN,1.000000,NaN,NaN,NaN,5.250000,NaN
59874,72145560801,NaN,NaN,1.000000,NaN,NaN,NaN,3.250000,NaN
59875,72145560900,NaN,NaN,1.000000,NaN,NaN,NaN,5.500000,NaN
59876,72153750400,NaN,NaN,1.000000,NaN,NaN,NaN,3.125000,NaN


We will bring over our previously calculated HRS from 'holc_rated' data set 

In [18]:
ai_merged= pd.merge(ai_long, holc_rated[['GEOID', 'HRS']], left_on='census_tract', right_on='GEOID',
    how='inner').drop(columns = ['GEOID'])
ai_merged.head(3)

,census_tract,asian_%approv,black_%approv,hisp_%approv,white_%approv,asian_interest,black_interest,hisp_interest,white_interest,HRS
0,01073000100,NaN,1.0,NaN,NaN,NaN,0.000,NaN,NaN,3.527723
1,01073000300,NaN,NaN,NaN,1.0,NaN,NaN,NaN,3.2185,3.999570
2,01073000400,NaN,1.0,NaN,NaN,NaN,3.875,NaN,NaN,3.777756


Calculate the correlation between interest rate and approval by race and HRS

In [19]:
#interest correlation
interest_corr=ai_merged[['HRS','census_tract', 'asian_interest', 'black_interest', 'hisp_interest',
       'white_interest', ]].corr().round(2).stack().reset_index()
#create long format to plot correlation 
interest_corr.rename(columns={0: 'corr_pearson', 'level_0': 'variable', 'level_1': 'variable2'}, inplace=True)

#approval percentage correlation
approval_corr=ai_merged[['HRS','census_tract', 'asian_%approv', 'black_%approv', 'hisp_%approv',
       'white_%approv', ]].corr().round(2).stack().reset_index()
#create long format to plot correlation 
approval_corr.rename(columns={0: 'corr_pearson', 'level_0': 'variable', 'level_1': 'variable2'}, inplace=True)

/tmp/ipykernel_539/358326168.py:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  'white_interest', ]].corr().round(2).stack().reset_index()
/tmp/ipykernel_539/358326168.py:9: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  'white_%approv', ]].corr().round(2).stack().reset_index()


We will also analyze  if there is other way HRS impacts  the loans profile, so we will calculate the correlation with loan amount, combined loan to value ratio, property value, debt to income ratio, tract minority population percent, discount points, tract to msa income percentage, approved

### Calculate correlation with loan amount. 

We will bring these variables from our cleaned 'df_hm_cleaned' dataset which originates from our loan data.  Filter out subset of variables for analysis and merge with 'holc_rated" data set 

In [20]:
tractDF =df_hm_cleaned.select('census_tract', 'loan_amount', 'combined_loan_to_value_ratio', 'property_value', 
    'tract_minority_population_percent', 'tract_to_msa_income_percentage', 'income', 'interest_rate' ).toPandas()

In [21]:
tractMerged= pd.merge(tractDF, holc_rated[['GEOID', 'HRS']], left_on='census_tract', right_on='GEOID',
    how='left').drop(columns = ['GEOID'])

#fill null values in HRS to create grade category
tractMerged['HRS'] = tractMerged['HRS'].fillna(0.1)
tractMerged['grade'] = pd.cut(tractMerged['HRS'], bins=[0,1, 1.75, 2.49, 3.3, 4], labels=['Ungraded', 'A','B', 
    'C', 'D'])


In [22]:
#convert variables to numeric values to calculate correlation. 
colToNumeric=['loan_amount', 'combined_loan_to_value_ratio', 'property_value', 'tract_minority_population_percent',
       'tract_to_msa_income_percentage', 'income', 'HRS']
tractMerged[colToNumeric]= tractMerged[colToNumeric].apply(pd.to_numeric, errors='coerce')


In [23]:
graded_corr= tractMerged[['loan_amount', 'combined_loan_to_value_ratio', 'property_value', 'tract_minority_population_percent',
       'tract_to_msa_income_percentage', 'income', 'HRS']].corr().round(2).stack().reset_index()
graded_corr.rename(columns={0: 'corr_pearson', 'level_0': 'variable', 'level_1': 'variable2'}, inplace=True)
graded_corr.head(3)

,variable,variable2,corr_pearson
0,loan_amount,loan_amount,1.00
1,loan_amount,combined_loan_to_value_ratio,0.06
2,loan_amount,property_value,0.85


### Plotting correlation.

We could not find a significant correlations between interest rate and approval percentage by race and HRS. We observe a minimal negative correlation between approval percentages for Black (0.01), Hispanic (-0.01) and White (-0.02) and HRS. 

In [24]:
#Pearson correlation matrix for interesr
basei=alt.Chart(interest_corr).mark_rect().transform_filter(
    alt.datum.variable < alt.datum.variable2
).encode(
    x='variable:O',
    y='variable2:O',
    color= 'corr_pearson:Q'
).properties(
    width=250,
    height=250,
    title=alt.TitleParams(
            text='Interest Rate ')
)




interest= basei+basei.mark_text().transform_calculate(label = '"" + datum.x + datum.y').encode(
    text='corr_pearson:N',
    color=alt.value('black'))

#Pearson correlation matrix for approvals
basea=alt.Chart(approval_corr).mark_rect().transform_filter(
    alt.datum.variable < alt.datum.variable2
).encode(
    x='variable:O',
    y= alt.Y('variable2:O', axis=None ),
    color= 'corr_pearson:Q'
).properties(
    width=250,
    height=250,
    title=alt.TitleParams(
            text='Approval Rate')
)




approval= basea+basea.mark_text().transform_calculate(label = '"" + datum.x + datum.y').encode(
    text='corr_pearson:N',
    color=alt.value('black'))


both=interest|approval
both.configure_title(fontSize=14).configure(background='#FFFFFF').configure_axis(
    grid=False)

/opt/conda/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.HConcatChart(...)

In [25]:
#Pearson correlation matrix for other variables
baseo=alt.Chart(graded_corr).mark_rect().transform_filter(
    alt.datum.variable < alt.datum.variable2
).encode(
    x='variable:O',
    y= alt.Y('variable2:O'),
    color= alt.Color('corr_pearson:Q', )
).properties(
    width=250,
    height=250,
    title=alt.TitleParams(
            text='HRS Correlation')
)




others= baseo+baseo.mark_text().transform_calculate(label = '"" + datum.x + datum.y').encode(
    text='corr_pearson:N',
    color=alt.value('black'))
others.configure_title(fontSize=14).configure(background='#FFFFFF').configure_axis(
    grid=False)

alt.LayerChart(...)

So far we seen that the percentage of minorities increases as it does HRS, as well as the number of vacancies. We did not discovered a significant relationship between interest rate by race and HRS. And only very minor negative relationship between HRS and approval rate by rate. 

When we calculate the correlation of HRS with other variables we discover a correlation with minority percentage, but near zero correlations with other variables of interest. Additionally, we identify strong correlations between: income, property value, loan amount and mean income percentage with reflect a normal relationship between individual income and the type of home they will be able to afford. 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=61770dc9-8282-488c-8a0c-8819ba3c4f95' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>